In [ ]:
from IPython.display import HTML, display
import csbgnpy.pd.io.sbgnml
import pypint
from tabulate import tabulate

## Merged map

From [1 - Merge CYCLE and CLOCK](1%20-%20Merge%20CYCLE%20and%20CLOCK.ipynb)

In [ ]:
merge_file = "generated/MERGE.sbgnml"
merge = csbgnpy.pd.io.sbgnml.read(merge_file)

## Phases and markers

From [2 - Markers](2%20-%20Markers.ipynb)

In [ ]:
%store -r clock_markers
%store -r clock_phases

To ids:

In [ ]:
markers = {phase: [merge.get_entity(e, by_string=True).id for e in ms]
               for phase, ms in clock_markers.items()}

## Initial state

From [3 - Initial state](3%20-%20Initial%20state.ipynb)

In [ ]:
%store -r inis

To ids:

In [ ]:
inis = {code: [e.id for e in ini] for code, ini in inis.items()}

## Dynamical models

In [ ]:
ans = [pypint.load(merge_file, initial_state=ini) for ini in inis.values()]

## A. Reach each phase

In [ ]:
results = []
for i, (code, ini) in enumerate(inis.items()):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    for phase in markers:
        res = []
        for marker in markers[phase]:
            res.append(an.reachability((marker, 1), fallback="mole"))
        res_ini.append("{}/{}".format(sum(res), len(res)))
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

In [ ]:
display(HTML(tabulate(results, headers=["Precursor state"] + list(markers.keys()), tablefmt="html")))

## B. Reach each phase with other phases to zero

In [ ]:
results = []
for i, (code, ini) in enumerate(inis.items()):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    for phase in markers:
        res = []
        for marker in markers[phase]:
            l = [(marker, 1)]
            for other_phase in markers:
                if other_phase != phase:
                    l += [(marker2, 0) for marker2 in markers[other_phase]]
            res.append(an.reachability(dict(l), reduce_for_goal=True, fallback="mole"))
        res_ini.append("{}/{}".format(sum(res), len(res)))
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

In [ ]:
display(HTML(tabulate(results, headers=["Precursor state"] + list(markers.keys()), tablefmt="html")))

## C. Reach each phase with previous phase(s) disabled

In [ ]:
results = []
couples = [(["RORG"], "SIRT1"), (["SIRT1"], "BMAL1-CLOCK"), (["RORG"], "BMAL1-CLOCK"), (["BMAL1-CLOCK"], "PER-CRY"), (["PER-CRY"], "RORG")]
for i, (code, ini) in enumerate(inis.items()):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    for phases_disabled, phase in couples:
        res = []
        an_disabled = an.disable([(marker, 1) for phase_disabled in phases_disabled for marker in markers[phase_disabled]])
        for marker in markers[phase]:
            res.append(an_disabled.reachability((marker, 1), fallback="mole"))
        res_ini.append("{}/{}".format(sum(res), len(res)))    
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

In [ ]:
display(HTML(tabulate(results, headers=["Precursor state"] + ["{} -> {}".format(", ".join(phases_disabled), phase) for phases_disabled, phase in couples], tablefmt="html")))

## D. Reach each phase with all markers to 1

In [ ]:
results = []
for i, (code, ini) in enumerate(inis.items()):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    seq = []
    for phase in markers:
        l = [(marker, 1) for marker in markers[phase]]
        res_ini.append(str(an.reachability(dict(l), reduce_for_goal=True, fallback="mole")))
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

In [ ]:
display(HTML(tabulate(results, headers=["Precursor state"] + list(markers.keys()), tablefmt="html")))

## E. Reach each phase sequentially with all markers to 1

In [ ]:
results = []
for i, (code, ini) in enumerate(inis.items()):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    seq = []
    for phase in markers:
        l = [(marker, 1) for marker in markers[phase]]
        seq.append(dict(l))
    res_ini.append(str(an.reachability(goal=pypint.Goal(*seq), reduce_for_goal=True, fallback="mole")))
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

In [ ]:
display(HTML(tabulate(results, headers=["Precursor state", "Reachable"], tablefmt="html")))

## F. Reach each phase sequentially with all markers to 1 and other phases to 0

In [ ]:
results = []
for i, (code, ini) in enumerate(inis.items()):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    seq = []
    for phase in markers:
        l = [(marker, 1) for marker in markers[phase]]
        for other_phase in markers:
            if other_phase != phase:
                l += [(marker2, 0) for marker2 in markers[other_phase]]
        seq.append(dict(l))
    res_ini.append(str(an.reachability(goal=pypint.Goal(*seq), reduce_for_goal=True, fallback="mole")))
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

In [ ]:
display(HTML(tabulate(results, headers=["Precursor state", "Reachable"], tablefmt="html")))

## G. Reach each phase sequentially with all markers to 1 and other phases to zero n times (n > 1)

In [ ]:
results = []
n = 4
for i, (code, ini) in enumerate(inis.items()):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    seq = []
    for phase in markers:
        l = [(marker, 1) for marker in markers[phase]]
        for other_phase in markers:
            if other_phase != phase:
                l += [(marker2, 0) for marker2 in markers[other_phase]]
        seq.append(dict(l))
    seq = seq * n
    res_ini.append(str(an.reachability(goal=pypint.Goal(*seq), reduce_for_goal=True, fallback="mole")))
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

In [ ]:
display(HTML(tabulate(results, headers=["Precursor state", "Reachable"], tablefmt="html")))